In [2]:
# Import required libraries
import wrds
import pandas as pd
import numpy as np

# WRDS Credentials 
WRDS_USERNAME = "saniazeb"

# Connect to WRDS database
db = wrds.Connection(wrds_username=WRDS_USERNAME)

# Define function to pull CDS data
def pull_cds_data(start_year=2001, end_year=2012):
    """
    Pulls 5-Year CDS data from WRDS Markit database for the given range of years.
    
    Returns:
        cds_data (DataFrame): Consolidated CDS data
    """
    all_data = []
    
    for year in range(start_year, end_year+1):
        table_name = f"markit.CDS{year}"
        query = f"""
        SELECT date, ticker, parspread
        FROM {table_name}
        WHERE tenor = '5Y' AND country = 'United States'
        """
        df = db.raw_sql(query, date_cols=['date'])
        all_data.append(df)
    
    cds_data = pd.concat(all_data, ignore_index=True)
    return cds_data

# Pull the CDS data
cds_data = pull_cds_data()

# Display first few rows
cds_data.head()


Loading library list...
Done


,date,ticker,parspread
0,2001-01-09,AA,0.004483
1,2001-01-09,AA,0.004192
2,2001-01-10,AA,0.004150
3,2001-01-10,AA,0.003880
4,2001-01-11,AA,0.004150


In [4]:
print(cds_data.columns)  # Check available column names
print(cds_data.head())   # Preview first few rows


Index(['ticker', 'parspread'], dtype='object')
                     ticker  parspread
date                                  
2001-01-02              ABS   0.012426
2001-01-02              AMR   0.016070
2001-01-02  AOL-TimeWarnInc   0.008174
2001-01-02              AVP   0.006171
2001-01-02              AXP   0.003252


In [6]:
import pandas as pd

# If 'date' is an index, reset it
if isinstance(cds_data.index, pd.DatetimeIndex):
    cds_data.reset_index(inplace=True)

# Ensure 'date' is in datetime format
cds_data['date'] = pd.to_datetime(cds_data['date'], errors='coerce')

# Drop rows where 'date' conversion failed
cds_data.dropna(subset=['date'], inplace=True)

# Ensure required columns exist
required_cols = {'date', 'ticker', 'parspread'}
missing_cols = required_cols - set(cds_data.columns)
if missing_cols:
    raise KeyError(f"Missing required columns: {missing_cols}")

# Group by 'date' and 'ticker', take the average spread
cds_data = cds_data.groupby(['date', 'ticker'], group_keys=False)['parspread'].mean().reset_index()

# Resample data to get **end-of-month values** using 'ME' instead of 'M'
cds_monthly = cds_data.set_index('date').groupby('ticker', group_keys=False).resample('ME').last().reset_index()

# Drop any rows with missing values after resampling
cds_monthly.dropna(inplace=True)

# Sort by date and parspread for quantile computation
cds_monthly = cds_monthly.sort_values(by=['date', 'parspread'])

# Assign quantiles (20 portfolios sorted by spread)
cds_monthly['quantile'] = cds_monthly.groupby('date')['parspread'].transform(
    lambda x: pd.qcut(x, 20, labels=False, duplicates='drop') + 1
)

# Drop any remaining missing quantile values
cds_monthly.dropna(subset=['quantile'], inplace=True)

# Convert quantile column to integer for correct pivoting
cds_monthly['quantile'] = cds_monthly['quantile'].astype(int)

# Ensure uniqueness in pivot table by taking the **median** in case of duplicates
cds_pivot = cds_monthly.pivot_table(index='date', columns='quantile', values='parspread', aggfunc='median')

# Rename columns to follow CDS_01, CDS_02, ..., CDS_20 naming convention
cds_pivot.columns = [f'CDS_{int(col):02d}' for col in cds_pivot.columns]

# Fill missing values using forward-fill to ensure smooth CDS spreads
cds_pivot.ffill(inplace=True)

# Reset index to keep 'date' as a column
cds_pivot.reset_index(inplace=True)

# Display processed data
print(cds_pivot.head())


        date    CDS_01    CDS_02    CDS_03    CDS_04    CDS_05    CDS_06  \
0 2001-01-31  0.002650  0.003450  0.004024  0.004411  0.005184  0.005576   
1 2001-02-28  0.002496  0.003038  0.004025  0.004381  0.005200  0.005729   
2 2001-03-31  0.002538  0.003216  0.003792  0.004077  0.004711  0.005462   
3 2001-04-30  0.002542  0.003135  0.003705  0.004127  0.004615  0.005179   
4 2001-05-31  0.002362  0.003000  0.003505  0.004000  0.004399  0.004833   

     CDS_07    CDS_08    CDS_09  ...    CDS_11    CDS_12    CDS_13    CDS_14  \
0  0.006131  0.006624  0.007332  ...  0.009014  0.010078  0.011173  0.012444   
1  0.006130  0.006942  0.007715  ...  0.009486  0.010559  0.011375  0.014500   
2  0.006066  0.006949  0.007750  ...  0.009158  0.010000  0.011000  0.012600   
3  0.005976  0.007129  0.007693  ...  0.009000  0.009612  0.010469  0.013000   
4  0.005814  0.006500  0.007197  ...  0.008260  0.009250  0.010575  0.012181   

     CDS_15    CDS_16    CDS_17    CDS_18    CDS_19    CDS_20 

In [13]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

# Define Loss Given Default (LGD) as 60%
LGD = 0.6

# Define start and end dates for the analysis
start_date = "2001-01-01"
end_date = "2012-12-31"

# ✅ Ensure 'date' is already the index, else reset it
if not isinstance(cds_pivot.index, pd.DatetimeIndex):
    raise KeyError("Expected 'date' as the index in cds_pivot, but it is missing.")

# ✅ Fetch & interpolate risk-free interest rates
def fetch_interest_rates():
    """
    Simulates fetching raw interest rate data (e.g., cash and swap rates).
    """
    # Example synthetic risk-free rates (Annualized Rates)
    rates = {
        "1M": 0.02, "3M": 0.022, "6M": 0.023, "1Y": 0.025, "2Y": 0.027, 
        "5Y": 0.03, "10Y": 0.035, "30Y": 0.04
    }
    tenors = np.array([1/12, 3/12, 6/12, 1, 2, 5, 10, 30])  # Convert months/years to fractional years
    rate_values = np.array(list(rates.values()))

    # Interpolate for quarterly tenors
    f = interp1d(tenors, rate_values, kind='linear', fill_value="extrapolate")
    quarterly_tenors = np.arange(0.25, 5.25, 0.25)  # 1Q to 20Q
    quarterly_rates = f(quarterly_tenors)

    # Convert to discount factors
    discount_factors = 1 / (1 + quarterly_rates / 4) ** (quarterly_tenors * 4)
    
    return pd.DataFrame(discount_factors, index=quarterly_tenors).T

# ✅ Get quarterly discount factors
quarterly_discount = fetch_interest_rates()

# ✅ Compute hazard rate lambda
lambda_df = 4 * np.log(1 + (cds_pivot / (4 * LGD)))

# ✅ Initialize Risky Duration DataFrame
quarters = range(1, 21)
risky_duration = pd.DataFrame(index=lambda_df.index, columns=lambda_df.columns)

# ✅ Compute survival probabilities and Risky Duration (RD)
for col in lambda_df.columns:
    quarterly_survival_probability = pd.DataFrame(index=lambda_df.index, columns=quarters)

    for quarter in quarters:
        quarterly_survival_probability[quarter] = np.exp(-((quarter * lambda_df[col]) / 4))

    # ✅ Compute Risky Duration using interpolated discount factors
    temp_df = quarterly_survival_probability * quarterly_discount.values
    risky_duration[col] = 0.25 * temp_df.sum(axis=1)

# ✅ Shift Risky Duration by 1 period to get RD_t-1
risky_duration_shifted = risky_duration.shift(1).ffill()

# ✅ Compute CDS Returns using the correct formula
cds_returns = (cds_pivot / 250) + (4 * cds_pivot.diff() * risky_duration_shifted)

# ✅ Fix Column Naming for CDS Returns
cds_returns.columns = [f'CDS_{int(col.split("_")[1]):02d}_RET' for col in cds_returns.columns]

# ✅ Reset index to keep 'date' in the dataset
cds_returns.reset_index(inplace=True)

# ✅ Display the final output
print(cds_returns.head())


        date  CDS_01_RET  CDS_02_RET  CDS_03_RET  CDS_04_RET  CDS_05_RET  \
0 2001-01-31         NaN         NaN         NaN         NaN         NaN   
1 2001-02-28   -0.002823   -0.007527    0.000037   -0.000539    0.000317   
2 2001-03-31    0.000783    0.003275   -0.004236   -0.005524   -0.008871   
3 2001-04-30    0.000083   -0.001473   -0.001587    0.000925   -0.001741   
4 2001-05-31   -0.003294   -0.002469   -0.003629   -0.002300   -0.003910   

   CDS_06_RET  CDS_07_RET  CDS_08_RET  CDS_09_RET  ...  CDS_11_RET  \
0         NaN         NaN         NaN         NaN  ...         NaN   
1    0.002797   -0.000008    0.005785    0.006934  ...    0.008492   
2   -0.004826   -0.001135    0.000152    0.000653  ...   -0.005826   
3   -0.005115   -0.001606    0.003279   -0.000995  ...   -0.002792   
4   -0.006271   -0.002909   -0.011327   -0.008892  ...   -0.013209   

   CDS_12_RET  CDS_13_RET  CDS_14_RET  CDS_15_RET  CDS_16_RET  CDS_17_RET  \
0         NaN         NaN         NaN        

In [6]:
import numpy as np
import pandas as pd
import wrds
from scipy.interpolate import interp1d
from pandas.tseries.offsets import MonthEnd

# WRDS Connection (Ensure WRDS Credentials are Set Up)
db = wrds.Connection()

# Define Loss Given Default (LGD)
LGD = 0.6

# Step 1: Pull CDS Data from WRDS
def get_cds_data(start_year=2001, end_year=2012):
    cds_data = []
    for year in range(start_year, end_year + 1):
        query = f"""
        SELECT date, ticker, parspread FROM markit.CDS{year}
        WHERE tenor = '5Y' AND country = 'United States'
        """
        temp_data = db.raw_sql(query, date_cols=['date'])
        cds_data.append(temp_data)
    return pd.concat(cds_data, ignore_index=True)

# Fetch CDS Spreads
cds_raw = get_cds_data()
cds_raw.set_index("date", inplace=True)
print("✅ Fetched CDS Data from WRDS")
display(cds_raw.head())

# Step 2: Process CDS Spreads & Assign Quantiles
def process_cds_spreads(cds_data):
    """
    Groups CDS spreads into 20 quantiles based on monthly spread distribution.
    """
    # Convert to monthly end
    cds_data = cds_data.groupby(['date', 'ticker'])['parspread'].mean().unstack()
    cds_data = cds_data.resample('ME').last().ffill()
    
    # Assign quantiles by sorting within each date
    def assign_quantiles(group):
        return pd.qcut(group.rank(method='first'), 20, labels=False, duplicates='drop') + 1
    
    cds_data = cds_data.stack().reset_index()
    cds_data.rename(columns={0: 'parspread'}, inplace=True)
    cds_data['quantile'] = cds_data.groupby('date')['parspread'].transform(assign_quantiles)
    
    # Compute portfolio average per quantile
    cds_pivot = cds_data.groupby(['date', 'quantile'])['parspread'].mean().unstack()
    
    # Rename columns to match expected format
    cds_pivot.columns = [f'CDS_{int(i):02d}' for i in range(1, 21)]
    return cds_pivot

# Process and Sort CDS into Quantiles
cds_spreads = process_cds_spreads(cds_raw)
print("✅ Processed CDS Spreads into Quantiles")
display(cds_spreads.head())

# Step 3: Compute Risky Duration (RD) Using Proper Discounting
lambda_df = 4 * np.log(1 + (cds_spreads / (4 * LGD)))
quarters = np.arange(1, 21)
risky_duration = pd.DataFrame(index=lambda_df.index, columns=lambda_df.columns)

for col in lambda_df.columns:
    quarterly_survival_probability = np.exp(-np.outer(quarters, lambda_df[col]) / 4).T
    risky_duration[col] = 0.25 * (quarterly_survival_probability.sum(axis=1))

# Apply Forward Fill to Risky Duration
risky_duration_shifted = risky_duration.shift(1).ffill()
cds_spread_shifted = cds_spreads.shift(1)
cds_spread_change = cds_spreads.diff()

# Step 4: Compute CDS Returns Using He-Kelly Formula (Strict Application)
cds_returns = (cds_spread_shifted / 250) + (cds_spread_change * risky_duration_shifted)

# Rename columns to match expected output format
cds_returns.columns = [f'CDS_{i:02d}_RET' for i in range(1, 21)]

# Step 5: Display and Save Results
print("✅ Computed CDS Returns Using He-Kelly Formula")
display(cds_returns.head())
output_dir = "./"
cds_returns.to_csv(output_dir + "computed_cds_returns.csv")
print("✅ CDS Returns Saved!")


Loading library list...
Done
✅ Fetched CDS Data from WRDS


,ticker,parspread
date,,
2001-01-09,AA,0.004483
2001-01-09,AA,0.004192
2001-01-10,AA,0.004150
2001-01-10,AA,0.003880
2001-01-11,AA,0.004150


✅ Processed CDS Spreads into Quantiles


,CDS_01,CDS_02,CDS_03,CDS_04,CDS_05,CDS_06,CDS_07,CDS_08,CDS_09,CDS_10,CDS_11,CDS_12,CDS_13,CDS_14,CDS_15,CDS_16,CDS_17,CDS_18,CDS_19,CDS_20
date,,,,,,,,,,,,,,,,,,,,
2001-01-31,0.002491,0.003348,0.003995,0.004357,0.005210,0.005587,0.006080,0.006660,0.007288,0.008138,0.008992,0.010081,0.011149,0.012384,0.014056,0.016156,0.018753,0.024871,0.032910,0.090619
2001-02-28,0.002401,0.003045,0.003904,0.004335,0.005036,0.005695,0.006140,0.006844,0.007637,0.008527,0.009553,0.010597,0.011713,0.014288,0.015456,0.017084,0.018973,0.025032,0.037200,0.109326
2001-03-31,0.002378,0.003109,0.003759,0.004163,0.004719,0.005417,0.006080,0.006960,0.007787,0.008347,0.009071,0.009990,0.011099,0.012498,0.014942,0.017035,0.018944,0.023875,0.034641,0.085254
2001-04-30,0.002376,0.003182,0.003746,0.004140,0.004618,0.005151,0.005919,0.007178,0.007727,0.008242,0.008989,0.009667,0.010624,0.013259,0.014943,0.017392,0.021215,0.025962,0.033924,0.083825
2001-05-31,0.002250,0.002962,0.003481,0.003885,0.004372,0.004843,0.005646,0.006441,0.007140,0.007683,0.008278,0.009294,0.010674,0.012207,0.014157,0.017051,0.020024,0.025238,0.033021,0.068202


✅ Computed CDS Returns Using He-Kelly Formula


,CDS_01_RET,CDS_02_RET,CDS_03_RET,CDS_04_RET,CDS_05_RET,CDS_06_RET,CDS_07_RET,CDS_08_RET,CDS_09_RET,CDS_10_RET,CDS_11_RET,CDS_12_RET,CDS_13_RET,CDS_14_RET,CDS_15_RET,CDS_16_RET,CDS_17_RET,CDS_18_RET,CDS_19_RET,CDS_20_RET
date,,,,,,,,,,,,,,,,,,,,
2001-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-02-28,-4.323589e-04,-0.001479,-0.000429,-0.000090,-0.000828,0.000550,0.000314,0.000918,0.001719,0.001911,0.002738,0.002513,0.002732,0.009074,0.006641,0.004390,0.001091,0.000822,0.018782,0.065201
2001-03-31,-1.043175e-04,0.000326,-0.000699,-0.000828,-0.001530,-0.001334,-0.000265,0.000592,0.000755,-0.000835,-0.002276,-0.002857,-0.002872,-0.008358,-0.002341,-0.000158,-0.000060,-0.005099,-0.010779,-0.077469
2001-04-30,9.566862e-08,0.000375,-0.000049,-0.000094,-0.000477,-0.001279,-0.000762,0.001082,-0.000257,-0.000471,-0.000357,-0.001508,-0.002218,0.003657,0.000063,0.001728,0.010542,0.009516,-0.002959,-0.004714
2001-05-31,-6.145411e-04,-0.001072,-0.001288,-0.001237,-0.001189,-0.001485,-0.001307,-0.003542,-0.002808,-0.002663,-0.003382,-0.001749,0.000282,-0.004914,-0.003621,-0.001514,-0.005351,-0.003135,-0.003774,-0.055212


✅ CDS Returns Saved!


In [3]:
from pathlib import Path
import pickle
import wrds
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import requests

# Define constants directly instead of using config module
OUTPUT_DIR = Path("output")
DATA_DIR = Path("data")
WRDS_USERNAME = "your_wrds_username"

def get_cds_data():
    print("Connecting to WRDS and fetching CDS data...")
    db = wrds.Connection(wrds_username=WRDS_USERNAME)
    cds_data = {}
    for year in range(2001, 2024):
        table_name = f"markit.CDS{year}"
        query = f"""
        SELECT date, ticker, parspread FROM {table_name} a 
        WHERE a.tenor = '5Y' AND a.country = 'United States'"""
        cds_data[year] = db.raw_sql(query, date_cols=['date'])
    print("CDS data fetching completed.")
    return cds_data

def assign_quantiles(group, n_quantiles=20):
    print("Assigning quantiles...")
    group['quantile'] = pd.qcut(group['parspread'], n_quantiles, labels=False) + 1
    return group

def resample_end_of_month(data):
    print("Resampling data to end of the month...")
    return data.resample('M').last()

def process_cds_data():
    print("Processing CDS data...")
    cds_data = pd.concat(get_cds_data().values(), axis=0)
    df = cds_data.groupby(['date', 'ticker']).mean().reset_index()
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    end_of_month_data = df.groupby('ticker').apply(resample_end_of_month)
    end_of_month_data.reset_index(level=0, drop=True, inplace=True)
    end_of_month_data.reset_index(inplace=True)
    end_of_month_data_sorted = end_of_month_data.sort_values(['date', 'parspread'])
    end_of_month_data_quantiled = end_of_month_data_sorted.groupby('date').apply(assign_quantiles)
    end_of_month_data_quantiled.reset_index(inplace=True)
    print("CDS data processing completed.")
    return end_of_month_data_quantiled

def calc_cds_monthly(method='median'):
    print(f"Calculating CDS monthly data using {method} method...")
    df = process_cds_data()
    df.set_index('quantile', inplace=True)
    
    def weighted_mean(data):
        weights = data['parspread']
        return (data['parspread'] * weights).sum() / weights.sum()
    
    if method == 'mean':
        comb_spread = df.groupby(['quantile', 'Date'])['parspread'].mean().reset_index()
    elif method == 'median':
        comb_spread = df.groupby(['quantile', 'Date'])['parspread'].median().reset_index()
    elif method == 'weighted':
        comb_spread = df.groupby(['quantile', 'Date']).apply(weighted_mean).reset_index(name='parspread')
    
    pivot_table = comb_spread.pivot_table(index='Date', columns='quantile', values='parspread')
    pivot_table.columns = [f'cds_{int(col)}' for col in pivot_table.columns]
    print("CDS monthly calculation completed.")
    return pivot_table

def process_cds_monthly(method='median'):
    print("Processing CDS monthly data...")
    df = calc_cds_monthly(method)
    mean, std = df['cds_20'].mean(), df['cds_20'].std()
    cutoff = std * 3
    lower, upper = mean - cutoff, mean + cutoff
    df['cds_20'] = df['cds_20'].rolling(window=15).median()
    print("CDS monthly processing completed.")
    return df

series_descriptions = {
    'DGS3MO': '3-Month Treasury Constant Maturity Rate',
    'DGS6MO': '6-Month Treasury Constant Maturity Rate',
}

def pull_fred_data(start_date, end_date, fred_series):
    print(f"Fetching FRED data from {start_date} to {end_date} for series: {fred_series}...")
    data = web.DataReader(fred_series, 'fred', start_date, end_date)
    print("FRED data fetching completed.")
    return data


In [4]:
import numpy as np
import pandas as pd
import wrds
from scipy.interpolate import interp1d
from pandas.tseries.offsets import MonthEnd

# WRDS Connection (Ensure WRDS Credentials are Set Up)
db = wrds.Connection()

# Define Loss Given Default (LGD)
LGD = 0.6

# Step 1: Pull CDS Data from WRDS
def get_cds_data(start_year=2001, end_year=2012):
    cds_data = []
    for year in range(start_year, end_year + 1):
        query = f"""
        SELECT date, ticker, parspread FROM markit.CDS{year}
        WHERE tenor = '5Y' AND country = 'United States'
        """
        temp_data = db.raw_sql(query, date_cols=['date'])
        cds_data.append(temp_data)
    return pd.concat(cds_data, ignore_index=True)

# Fetch CDS Spreads
cds_raw = get_cds_data()
cds_raw.set_index("date", inplace=True)
print("\u2705 Fetched CDS Data from WRDS")
display(cds_raw.head())

# Step 2: Process CDS Spreads & Assign Quantiles
def process_cds_spreads(cds_data):
    """
    Groups CDS spreads into 20 quantiles based on monthly spread distribution.
    """
    # Convert to monthly end
    cds_data = cds_data.groupby(['date', 'ticker'])['parspread'].mean().unstack()
    cds_data = cds_data.resample('ME').last().ffill()
    
    # Assign quantiles by sorting within each date
    def assign_quantiles(group):
        return pd.qcut(group.rank(method='first'), 20, labels=False, duplicates='drop') + 1
    
    cds_data = cds_data.stack().reset_index()
    cds_data.rename(columns={0: 'parspread'}, inplace=True)
    cds_data['quantile'] = cds_data.groupby('date')['parspread'].transform(assign_quantiles)
    
    # Compute portfolio average per quantile
    cds_pivot = cds_data.groupby(['date', 'quantile'])['parspread'].mean().unstack()
    
    # Rename columns to match expected format
    cds_pivot.columns = [f'CDS_{int(i):02d}' for i in range(1, 21)]
    return cds_pivot

# Process and Sort CDS into Quantiles
cds_spreads = process_cds_spreads(cds_raw)
print("\u2705 Processed CDS Spreads into Quantiles")
display(cds_spreads.head())

# Step 3: Compute Risky Duration (RD) Using Proper Discounting
lambda_df = np.log(1 + (cds_spreads / (LGD)))
quarters = np.arange(1, 21)
risky_duration = pd.DataFrame(index=lambda_df.index, columns=lambda_df.columns)

for col in lambda_df.columns:
    quarterly_survival_probability = np.exp(-np.outer(quarters, lambda_df[col])).T
    risky_duration[col] = 0.25 * (quarterly_survival_probability.sum(axis=1))

# Apply Forward Fill to Risky Duration
risky_duration_shifted = risky_duration.shift(1).ffill()
cds_spread_shifted = cds_spreads.shift(1)
cds_spread_change = cds_spreads.diff()

# Step 4: Compute CDS Returns Using He-Kelly Formula (Strict Application)
cds_returns = (cds_spread_shifted / 250) + (cds_spread_change * risky_duration_shifted)

# Rename columns to match expected output format
cds_returns.columns = [f'CDS_{i:02d}_RET' for i in range(1, 21)]

# Step 5: Display and Save Results
print("\u2705 Computed CDS Returns Using He-Kelly Formula")
display(cds_returns.head())
output_dir = "./"
cds_returns.to_csv(output_dir + "computed_cds_returns.csv")
print("\u2705 CDS Returns Saved!")


Loading library list...
Done
✅ Fetched CDS Data from WRDS


,ticker,parspread
date,,
2001-01-09,AA,0.004483
2001-01-09,AA,0.004192
2001-01-10,AA,0.004150
2001-01-10,AA,0.003880
2001-01-11,AA,0.004150


✅ Processed CDS Spreads into Quantiles


,CDS_01,CDS_02,CDS_03,CDS_04,CDS_05,CDS_06,CDS_07,CDS_08,CDS_09,CDS_10,CDS_11,CDS_12,CDS_13,CDS_14,CDS_15,CDS_16,CDS_17,CDS_18,CDS_19,CDS_20
date,,,,,,,,,,,,,,,,,,,,
2001-01-31,0.002491,0.003348,0.003995,0.004357,0.005210,0.005587,0.006080,0.006660,0.007288,0.008138,0.008992,0.010081,0.011149,0.012384,0.014056,0.016156,0.018753,0.024871,0.032910,0.090619
2001-02-28,0.002401,0.003045,0.003904,0.004335,0.005036,0.005695,0.006140,0.006844,0.007637,0.008527,0.009553,0.010597,0.011713,0.014288,0.015456,0.017084,0.018973,0.025032,0.037200,0.109326
2001-03-31,0.002378,0.003109,0.003759,0.004163,0.004719,0.005417,0.006080,0.006960,0.007787,0.008347,0.009071,0.009990,0.011099,0.012498,0.014942,0.017035,0.018944,0.023875,0.034641,0.085254
2001-04-30,0.002376,0.003182,0.003746,0.004140,0.004618,0.005151,0.005919,0.007178,0.007727,0.008242,0.008989,0.009667,0.010624,0.013259,0.014943,0.017392,0.021215,0.025962,0.033924,0.083825
2001-05-31,0.002250,0.002962,0.003481,0.003885,0.004372,0.004843,0.005646,0.006441,0.007140,0.007683,0.008278,0.009294,0.010674,0.012207,0.014157,0.017051,0.020024,0.025238,0.033021,0.068202


✅ Computed CDS Returns Using He-Kelly Formula


,CDS_01_RET,CDS_02_RET,CDS_03_RET,CDS_04_RET,CDS_05_RET,CDS_06_RET,CDS_07_RET,CDS_08_RET,CDS_09_RET,CDS_10_RET,CDS_11_RET,CDS_12_RET,CDS_13_RET,CDS_14_RET,CDS_15_RET,CDS_16_RET,CDS_17_RET,CDS_18_RET,CDS_19_RET,CDS_20_RET
date,,,,,,,,,,,,,,,,,,,,
2001-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-02-28,-4.182831e-04,-0.001416,-0.000407,-0.000084,-0.000773,0.000513,0.000293,0.000846,0.001569,0.001727,0.002448,0.002219,0.002382,0.007786,0.005592,0.003614,0.000885,0.000638,0.012965,0.029469
2001-03-31,-1.008190e-04,0.000314,-0.000664,-0.000783,-0.001433,-0.001238,-0.000244,0.000545,0.000689,-0.000747,-0.002014,-0.002497,-0.002476,-0.006999,-0.001926,-0.000115,-0.000032,-0.003773,-0.007070,-0.031428
2001-04-30,3.821790e-07,0.000361,-0.000046,-0.000088,-0.000448,-0.001192,-0.000703,0.000993,-0.000230,-0.000421,-0.000315,-0.001326,-0.001925,0.003138,0.000062,0.001417,0.008401,0.007200,-0.001958,-0.001997
2001-05-31,-5.955708e-04,-0.001028,-0.001226,-0.001172,-0.001120,-0.001389,-0.001210,-0.003230,-0.002543,-0.002396,-0.003015,-0.001544,0.000252,-0.004162,-0.003004,-0.001212,-0.004133,-0.002286,-0.002528,-0.025577


✅ CDS Returns Saved!
